In [1]:
import json
from transformers import AutoTokenizer
import torch
from torch.utils.data import DataLoader

/home/sangdt/research/JointBert/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [155]:
from transformers import modeling_roberta

ImportError: cannot import name 'modeling_roberta' from 'transformers' (/home/sangdt/research/JointBert/venv/lib/python3.8/site-packages/transformers/__init__.py)

In [2]:
data_test = []
with open("/home/sangdt/research/intent-entity-bert/entity_recognitor/data/test_slots.json") as f:
    for line in f:
        data_test.append(json.loads(line))

In [3]:
with open('/home/sangdt/research/JointBert/data/test_slots.jsonl', 'w') as outfile:
    for entry in data_test:
        json.dump(entry, outfile)
        outfile.write('\n')

In [2]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [3]:
tokenizer("alo", )

{'input_ids': [0, 7747, 2], 'attention_mask': [1, 1, 1]}

In [12]:
distinct_intents = ['activation', 'agent', 'ask_info', 'barring', 'cancellation', 'change',
 'connection', 'continue', 'downloading', 'exit', 'greetings', 'help', 'leaving',
 'no_intention', 'ordering', 'payment', 'purchase', 'refunding' ,'register',
 'renewal' ,'repairing' ,'repeat' ,'report_issues', 'returning' ,'start',
 'top-up', 'transferring', 'unblocking', 'unlocking', 'upgrading']

In [10]:
len(distinct_slots)

141

In [68]:
tokenizer.tokenize("<s>customer service</s>")

['<s>', 'custom', 'er', 'Ġservice', '</s>']

In [69]:
vocab = tokenizer.get_vocab()

In [70]:
def modify_slot(list_slots, sentence):
    list_bert_tokens = tokenizer.tokenize(sentence.strip())
    attention_mask = tokenizer(sentence.strip(), add_special_tokens=True)['attention_mask']
    list_index_g = [index for index, i in enumerate(list_bert_tokens) if 'Ġ' in i]
    list_index_g = [0] + list_index_g
    new_list_slots = []
    for index, token in enumerate(list_bert_tokens):
        if index not in list_index_g:
            tmp = [i for i in list_index_g if i < index][-1]
            ner_tag = list_slots[list_index_g.index(tmp)].replace("B-", "I-")
        else:
            ner_tag = list_slots[list_index_g.index(index)]
        new_list_slots.append(ner_tag)
    list_ids_tokens = [vocab[i] for i in list_bert_tokens]
    return {"token_ids": [0] + list_ids_tokens + [2], "labels": ['O'] +  new_list_slots + ['O'], "attention_mask": attention_mask}

In [149]:
def transform(sentences, list_slots):
    new_list_slots = []
    list_ids_slots = []
    list_pre = tokenizer(sentences, padding=True, truncation=True, add_special_tokens=True)
    for i in range(len(sentences)):
        list_tokens = tokenizer.convert_ids_to_tokens(list_pre['input_ids'][i])
        list_index_g = [index for index, i in enumerate(list_tokens) if 'Ġ' in i]
        list_index_g = [1] + list_index_g
        new_slots = []
        for index, token in enumerate(list_tokens):
            if index not in list_index_g:
                if token in ['<s>', '</s>', '<pad>']:
                    ner_tag = 'O'
                else:
                    tmp = [i for i in list_index_g if i < index][-1]
                    ner_tag = list_slots[i][list_index_g.index(tmp)].replace("B-", "I-")
            else:
                ner_tag = list_slots[i][list_index_g.index(index)]
            new_slots.append(ner_tag)
        new_list_slots.append(new_slots)

    for slots in new_list_slots:
        list_ids_slots.append([dict_tags[i] for i in slots])
    return (
        torch.tensor(list_pre['input_ids']),
        torch.tensor(list_pre['attention_mask']),
        torch.tensor(list_ids_slots)
    )


In [108]:
torch.tensor([[1, 2, 3], [1,2,3]], dtype=   )

tensor([[1, 2, 3],
        [1, 2, 3]])

In [141]:
def collate_fn(batch):
    sentences = [example['text'] for example in batch]
    list_slots = [example['slots'] for example in batch]
    return transform(sentences, list_slots)

In [78]:
new_labels = list(set([example for i in distinct_slots for example in [i, i.replace("B-", "I-")]]))

In [105]:
dict_tags = {tag: i for i, tag in enumerate(new_labels)}

In [147]:
dict_tags['B-CUSTOMER']

203

In [13]:
with open('/home/sangdt/research/JointBert/processed_data/intent_label.txt', 'w') as f:
    for line in distinct_intents:
        f.write(f"{line}\n")

In [71]:
new_data_train = []
for example in data_train:
    new_data_train.append(modify_slot(example['slots'], example['text']))

In [72]:
len(new_data_train)

4042

In [73]:
print(data_train[13])

{'text': 'customer services please', 'tokens': ['customer', 'services', 'please'], 'intention': 'agent', 'slots': ['B-CUSTOMER', 'B-SERVICE', 'O']}


In [74]:
print(new_data_train[13])

{'token_ids': [0, 31458, 254, 518, 2540, 2], 'labels': ['O', 'B-CUSTOMER', 'I-CUSTOMER', 'B-SERVICE', 'O', 'O'], 'attention_mask': [1, 1, 1, 1, 1, 1]}


In [91]:
tokenizer.tokenize(('a encoder', 'bi encoder'))

['a', 'Ġenc', 'oder', 'bi', 'Ġenc', 'oder']

In [96]:
tokenizer.convert_ids_to_tokens([0, 31458, 254, 518, 2540, 2])

['<s>', 'custom', 'er', 'Ġservices', 'Ġplease', '</s>']

In [144]:
tokenizer(['aa', 'bb', 'cc dd'], padding=True)

{'input_ids': [[0, 6621, 2, 1], [0, 14141, 2, 1], [0, 7309, 47821, 2]], 'attention_mask': [[1, 1, 1, 0], [1, 1, 1, 0], [1, 1, 1, 1]]}

In [104]:
vocab['<pad>']

1

In [150]:
data_loader = DataLoader(data_train, batch_size=16, collate_fn=collate_fn)

In [151]:
for i in data_loader:
    print(i)
    break

(tensor([[    0, 31458,   254,   544,     2,     1,     1,     1,     1],
        [    0,   858, 15027,     2,     1,     1,     1,     1,     1],
        [    0,  2362,  3392,    47,     2,     1,     1,     1,     1],
        [    0,  4651,  1355,     2,     1,     1,     1,     1,     1],
        [    0, 39397,     7,  2111,   544,     2,     1,     1,     1],
        [    0,  2558,  2753,  1355,     2,     1,     1,     1,     1],
        [    0,   560,  1994,     7,  2111,   544,     2,     1,     1],
        [    0, 11131,     2,     1,     1,     1,     1,     1,     1],
        [    0,  2362,  2888,  2748,     2,     1,     1,     1,     1],
        [    0, 31458,   254,   544,  2540,     2,     1,     1,     1],
        [    0, 28825,  1052,     2,     1,     1,     1,     1,     1],
        [    0,   560,  1067,     7,  2111,   544,     2,     1,     1],
        [    0, 28023,     2,     1,     1,     1,     1,     1,     1],
        [    0, 31458,   254,   518,  2540,     2,

In [154]:
vocab['<cls>']

KeyError: '<cls>'

In [1]:
import numpy as np


In [2]:
tmp = np.array([[1, 2, 3]])

In [3]:
tmp.shape

(1, 3)

In [5]:
np.append(tmp, [[1, 2, 3,]], axis=0).tolist()

[[1, 2, 3], [1, 2, 3]]